# Synacor Challenge VM

**Solution Started**: July 7, 2025  
**Author**: Abbas Moosajee

This notebook loads and runs the Synacor challenge binary using a custom `VirtualMachine` implementation, highlights challenge codes, and displays the interactive game output in styled HTML.

In [41]:
import os, re, time
from html import escape
from IPython.display import display, HTML
from VirtualMachine import VirtualMachine
start_time = time.time()

In [42]:
# Load the binary program file
program_file = "challenge.bin"
file_path = os.path.join(os.getcwd(), program_file)  # assumes file in current working dir
with open(file_path, "rb") as f:
    vm_program = f.read()

In [43]:
class SynacorConsole:
    def __init__(self, software):
        self.console = VirtualMachine(software)
        self.code_no = 0
        self.challenge_codes = {}

    @staticmethod
    def md5_hash(code: str) -> str:
        import hashlib
        return hashlib.md5(code.encode('utf-8')).hexdigest()

    def get_challenge_codes(self, text):
        """Extract and store all mixed-case challenge codes."""
        all_codes = re.findall(
            r"\b((?:[a-z]+[A-Z]+[a-z]|[A-Z]+[a-z]+[A-Z])\w*)\b", text
        )
        for code in all_codes:
            self.code_no += 1
            hash_result = self.md5_hash(code)
            self.challenge_codes[self.code_no] = (code, hash_result)
        return all_codes

    def format_terminal(self, text, codes_color):
        """Highlight mixed-case challenge codes in HTML."""
        text = escape(text)
        return re.sub(
            r"\b((?:[a-z]+[A-Z]+[a-z]|[A-Z]+[a-z]+[A-Z])\w*)\b",
            rf"<span style='color:{codes_color}'>\1</span>",
            text
        )

    def display_terminal(self, text):
        """Format and display terminal output with highlighted codes."""
        self.color_dict = {
            "background": "#282828",
            "terminal": "#33FF00",
            "codes": "#FFB000"
        }
        formatted = text.replace('. ', '.\n')
        html = (
            f"<div style='background-color: {self.color_dict['background']}; padding: 1ex'>"
            f"<pre style='background-color: {self.color_dict['background']}; color: {self.color_dict['terminal']}; margin: 0'>"
            + self.format_terminal(formatted, self.color_dict['codes'])
            + "</pre></div>"
        )
        display(HTML(html))

    def play_game(self, actions = []):
        full_terminal = self.console.run_computer()
        current_terminal = full_terminal[-1]
        self.get_challenge_codes(current_terminal)
        self.display_terminal(current_terminal)

In [44]:
# Run the VM with simulated input
console = SynacorConsole(vm_program)
console.play_game()

In [45]:
print(f"Execution Time = {time.time() - start_time:.5f}s")

Execution Time = 1.40084s
